<a href="https://colab.research.google.com/github/Krish7Hack/Side-Channel-Attack-Resistant-Cryptographic-Libraries/blob/main/Side_Channel_Attack_Resistant_Cryptographic_Libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Side-Channel Attack Resistant Cryptographic Libraries
# =====================================================
# This notebook implements constant-time versions of AES, RSA, and ECC,
# and evaluates their resistance to timing side-channel attacks.


In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import secrets
import random
from concurrent.futures import ThreadPoolExecutor

# Helper functions for timing measurements

In [2]:
def measure_execution_time(func,*args,**kwargs):
  "Measure execution time of a function in nanoseconds"
  start = time.perf_counter_ns() #Time spent during sleep or system clock cahnges
  result = func(*args,**kwargs)
  end = time.perf_counter_ns() #Records the time after the functions finishes executing
  return result,end-start

In [3]:
def collect_timing_samples(func,samples=1000,*args,**kwargs):
  "Collect timing sample for each function"
  times = []
  for _ in tqdm(range(samples)):
    _, execution_time = measure_execution_time(func,*args,**kwargs)
    times.append(execution_time)
  return times
#Here tqdm shows a smart progress bar for loop and iterable operations

In [4]:
def plot_timing_comparison(timing_data,labels,title):
  "Plot timing comparison between implementation"
  plt.figure(figsize=(12,6))

  for i,times in enumerate(timing_data):
    plt.hist(times,bins=50,alpha=0.5,label=labels[i])

    plt.xlabel('Execution Time (ns)')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend()
    plt.grid(True,alpha=0.3)
    plt.show()

    #Print Statistics
    for i ,times in enumerate(timing_data):
      print(f"{labels[i]}")
      print(f" Mean: {np.mean(times):.2f} ns")
      print(f"  Std Dev: {np.std(times):.2f} ns")
      print(f"  Variance: {np.var(times):.2f}")
      print(f"  Min: {np.min(times):.2f} ns")
      print(f"  Max: {np.max(times):.2f} ns")
      print()

# 1. Constant-Time AES Implementation


In [5]:
class AES:
  """ A constant time implementation of AES-128 """
  #AES S-BOX
  SBOX = [
      0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
      0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
      0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
      0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
      0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
      0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
      0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
      0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
      0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
      0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
      0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
      0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
      0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
      0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
      0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
      0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
    ]
  #AES inverse S-BOX
  INV_SBOX = [
      0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
      0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
      0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
      0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
      0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
      0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
      0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
      0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
      0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
      0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
      0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
      0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
      0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
      0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
      0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
      0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d
    ]
  #Round Constants
  RCON = [0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]

In [6]:
def __init__(self,key):
  "Initialize with a 16-byte key"
  if len(key) != 16:
    raise ValueError("Key must be 16 bytes (128bits)")
  self.round_keys = self._key_expansion(key)

In [7]:
def _sub_bytes(self,state):
  """SubBytes transformation - constant time implementation"""
  for i in range(16):
    state[i] = self.SBOX[state[i]]
  return state

In [8]:
def _shift_rowa(self,state):
  """ShiftRows Transformation"""
  # Convert state to 4 x 4 matrix
  matrix = [state[i::4] for i in range(4)]

  #Shift rows
  for i in range(1,4):
    matrix[i] = matrix[i][i:] + matrix[i][:i]
  #Convert back to flat array
  result = []
  for col in range(4):
    for row in range(4):
      result.append(matrix[row][col])

  return result

In [9]:
def _mix_columns(self,state):
  """MixColumns transformation """
  result = [0] * 16
  for col in range(4):
    s0 = state[col*4]
    s1 = state[col*4 + 1]
    s2 = state[col*4 + 2]
    s3 = state[col*4 + 3]

    #Constant-time Galois field multiplication
    result[col * 4] = self._gmul(2,s0) ^ self._gmul(3,s1) ^ s2 ^ s3
    result[col * 4 + 1] = s0 ^ self._gmul(2,s1) ^ self.gmul(3,s2) ^ s3
    result[col * 4 + 2] = s0 ^ s1 ^ self._gmul(2,s2) ^ self.gmul(3,s3)
    result[col * 4 + 3] = self.gmul(3,s0) ^ s1 ^ s2 ^ self.gmul(2,s3)

  return result

In [ ]:
def _gmul(self,a,b):
  """Galois field multiplication in constant time"""
  p = 0
  for i in range(8):
    if b & 1:
      p ^= a

    #Track if a will overflow
    high_bit = a & 0x80

    #Left shift a
    a <<= 1

    #If there was a high bit,XOR with 0x1B
    if high_bit:
      a ^= 0x1B

    #Right shit b
    b >>= 1

  return p & 0xFF

In [ ]:
def _add_round_key(self,state,round_key):
  """Add RoundKey transformation """
  for i in range(16):
    state[i] ^= round_key[i]
  return state

In [ ]:
def _key_expansion(self,key):
  """Key expansion algorithm"""
  key_bytes = list(key)
  w = [0] * 176 #11 round keys(16 bytes each)

  #Copy the initial key
  for i in range(16):
    w[i] = key_bytes[i]

  #Generate the expanded key
  for i in range(4,44):
    temp = w[(i-1)*4:(i-1)*4+4]

    if i % 4 == 0:
      #Rotword : Rotate Left
      temp = temp[1:] + temp[:1]

      #Subword : Apply S-box
      for j in range(4):
        temp[j] = self.Sbox[temp[j]]